# 使用RNN生产文本
[教程出处](https://www.tensorflow.org/tutorials/text/text_generation)

In [1]:
import tensorflow as tf
import numpy as np
import os
import time

/Users/mac/.jackprogram/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/mac/.jackprogram/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/mac/.jackprogram/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/mac/.jackprogram/anaconda3/lib/python3.7/s

In [2]:
# 下载训练数据
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 8s 7us/step


# 了解训练数据

In [4]:
# 读文件, 变成py2的格式
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length就是有好多个字符串
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [5]:
# 看看前面的250个字符
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [6]:
# 我们看看有多少个不一样的字符
vocab = sorted(set(text)) # vocab里面存了不同的字符
print ('{} unique characters'.format(len(vocab)))

65 unique characters


# 处理数据

## 向量化文本
创建两个查找表: 字符👉数字 和 数字👉字符

In [14]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

In [18]:
# 输入任意一串字符那么就都可以表示成一串数字了
text_as_int = np.array([char2idx[c] for c in text])

array([18, 47, 56, ..., 45,  8,  0])

In [19]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
  ...
}


In [20]:
# 展示是如何映射的
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'First Citizen' ---- characters mapped to int ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


# 我们的目标
给一串字符，最可能的下一个字符是什么？

## 制作训练样本和目标